In [37]:
import pandas as pd
import numpy as np
import PyPDF2
import textract
import pdfplumber
import base64
import re
import io
from collections import Counter


In [38]:
'''
#Basic Code Snippet that is used to extract structure of the pdf
#From this we identify keywords with / key before them using the regex.
#Uncomment and run this to see the raw pdf data

import base64
import re
import io

filename = 'MaliciousPDF.pdf'
filenameV2 = 'MaliciousPDF.txt'
encoding = 'utf-8'
regex = '\/[^\s\n\r]+\s'

with open(filename, mode="r",encoding='utf-8',errors='ignore') as pdf_file:
    encoded_string = pdf_file.readlines()
    print(encoded_string)
    
    with io.open(filenameV2, "w", encoding="utf-8") as f:
        f.writelines(encoded_string)

'''

'\n#Basic Code Snippet that is used to extract structure of the pdf\n#From this we identify keywords with / key before them using the regex.\n#Uncomment and run this to see the raw pdf data\n\nimport base64\nimport re\nimport io\n\nfilename = \'MaliciousPDF.pdf\'\nfilenameV2 = \'MaliciousPDF.txt\'\nencoding = \'utf-8\'\nregex = \'\\/[^\\s\n\r]+\\s\'\n\nwith open(filename, mode="r",encoding=\'utf-8\',errors=\'ignore\') as pdf_file:\n    encoded_string = pdf_file.readlines()\n    print(encoded_string)\n    \n    with io.open(filenameV2, "w", encoding="utf-8") as f:\n        f.writelines(encoded_string)\n\n'

In [52]:
#Here we use regex to find all possible keywords and 
#then return the ones without any special character or digit

regex = '(endobj|obj|endstream|stream|startxref|xref|trailer|\/[^\s\n\r]+)'

def FilteringFunction(match):
    return match[1:].isalnum()

def FilteringFunctionOnlyAlphabets(match):
    return match[1:].isalpha()

def GetKeywords(fileNamePDF) :
    with open(filename, mode="r",encoding='utf-8',errors='ignore') as pdf_file:
        encoded_string = pdf_file.read()
        matches = re.findall(regex, encoded_string) 
        filteredMatches = list(filter(FilteringFunctionOnlyAlphabets, matches))
        
        #print(filtedMatches)
        #print(len(filteredMatches))
        
        return filteredMatches

def weightage(number_of_times_word_appeared, textLength, number_of_documents=1):
    
    tf = number_of_times_word_appeared/float(textLength)
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return tf,idf ,tf_idf

def GetKeywordVector(matches, textLength):
    matchesCount = Counter(matches)
    data_items = matchesCount.items()
    data_list = list(data_items)
    df = pd.DataFrame(data_list)
    
    df['tf'] = df[1].apply(lambda x: weightage(x,textLength)[0])
    df['idf'] = df[1].apply(lambda x: weightage(x,textLength)[1])
    df['tf_idf'] = df[1].apply(lambda x: weightage(x,textLength)[2])
    df = df.sort_values('tf_idf',ascending=False)
    
    print(df)
    
fileName = 'MaliciousPDF.pdf'
fileNameTxt = 'MaliciousPDF.txt'

matches = GetKeywords(fileName)
GetKeywordVector(matches, len(encoded_string))

1697
            0    1        tf       idf    tf_idf
129       /zd    1  0.000085  0.000000  0.000000
195        /h    1  0.000085  0.000000  0.000000
165      /Wzp    1  0.000085  0.000000  0.000000
166     /lBbC    1  0.000085  0.000000  0.000000
167       /Sr    1  0.000085  0.000000  0.000000
..        ...  ...       ...       ...       ...
5     /Length   49  0.004179 -3.891820 -0.016264
17   /XObject   51  0.004350 -3.931826 -0.017102
10      /Type   89  0.007591 -4.488636 -0.034072
4      endobj  161  0.013731 -5.081404 -0.069775
0         obj  161  0.013731 -5.081404 -0.069775

[258 rows x 5 columns]
